In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%load_ext cython

In [3]:
import ROOT
import h5py
import os
from matplotlib import pyplot as plt
import numpy as np
import time
import sys
import datetime
import time

ROOT.gSystem.Load("$MARSSYS/libmars.so")

/home/iwsatlas1/yusuke/software/root_v5.34.36/lib/ROOT.py:352: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return _orig_ihook( name, *args, **kwds )


0

In [4]:
#filepath_in = "/remote/ceph/user/m/mhuetten/MAGIC_data/sorcerer/"
#filename_M1 = "20180821_M1_05074767.014_Y_CrabNebula-W0.40+035.root"
##filename_M2 = "20180821_M2_05074767.014_D_CrabNebula-W0.40+035.root"

filepath_in  = "/remote/ceph/user/y/yusuke/cta/MAGIC_LST/"
filepath_out = "/remote/ceph/user/y/yusuke/cta/MAGIC_LST/"

# Observation data files
#filename_M1 = "20180821_M1_05074760.001_Y_CrabNebula-W0.40+215.root"
#filename_M2 = "20180821_M2_05074760.001_Y_CrabNebula-W0.40+215.root"

# Monte Carlo files
filename_M1 = "GA_M1_za05to35_9_188880_Y_w0.root"
filename_M2 = "GA_M2_za05to35_9_188880_Y_w0.root"

ntels = 2

camgeom = ROOT.MGeomCamMagicTwo()
#print camgeom.GetPixels().At(4).Print() 

camera = ROOT.MHCamera(camgeom)

filenames = [filename_M1, filename_M2]
filepaths = [os.path.join(filepath_in, filename_M1), os.path.join(filepath_in, filename_M2)]

infile_M1 = ROOT.TFile(os.path.join(filepath_in, filename_M1))
infile_M2 = ROOT.TFile(os.path.join(filepath_in, filename_M2))

infiles = [infile_M1, infile_M2]

filename_out = ''.join(filename_M1.split("_M1"))

In [5]:
#ROOT.TBrowser()

In [6]:
def get_ROOT_array(file, tree, branch, leaf):
    tfile = ROOT.TFile(file)
    ttree = getattr(tfile, tree)
    hist = ROOT.TH1D("hist","hist",100,-100,100)
    ttree.Project("hist", "%s.%s"%(branch,leaf),"","",100000000000,0)
    treeArr = ttree.GetV1()
    result = np.array(np.frombuffer(treeArr,count=int(hist.GetEntries())), dtype = np.double)
    return result

In [7]:
f = h5py.File(os.path.join(filepath_out,filename_out.split(".root")[0] + ".hdf5"),"w")
dsets_events = []

for ntel in range(ntels):
    
    Runheader = ROOT.TChain("RunHeaders")
    Runheader.Add(os.path.join(filepath_in, filenames[ntel]))

    head = ROOT.MRawRunHeader()
    Runheader.SetBranchAddress("MRawRunHeader.",head)

    # extract global information out of first event:
    Runheader.GetEntry(0)
    TelescopeNumber =  head.GetTelescopeNumber()
    nevents = head.GetNumEvents()
    nevents = 1
    #max_event = Events.GetEntries()
    nevents_check = 0
    
    RunNumber = head.GetRunNumber()
    RunType = head.GetRunTypeStr()
    if RunType == "Data":
        is_mc = False
    elif RunType == "Monte Carlo":
        is_mc = True
        Core_x_vec = get_ROOT_array(filepaths[ntel], "Events", "MMcEvt", "fCoreX")
        Core_y_vec = get_ROOT_array(filepaths[ntel], "Events", "MMcEvt", "fCoreY")
        h_first_vec = get_ROOT_array(filepaths[ntel], "Events", "MMcEvt", "fZFirstInteraction")
    else:
        is_mc = False
    
    Events = ROOT.TChain("Events")
    Events.Add(os.path.join(filepath_in, filenames[ntel]))
    
    evtdata_phe = ROOT.MCerPhotEvt()
    evtdata_arrival = ROOT.MArrivalTime()

    evtheader = ROOT.MRawEvtHeader()
    if is_mc == True:
        mcevtheader = ROOT.MMcEvt()
 
    Events.SetBranchAddress("MCerPhotEvt.",evtdata_phe)
    Events.SetBranchAddress("MArrivalTime.",evtdata_arrival)
    Events.SetBranchAddress("MRawEvtHeader.",evtheader)
    if is_mc == True:
        Events.SetBranchAddress("MMcEvt.",mcevtheader)
    
    arr_phe_2D = []
    arr_arrival_2D = []
    arr_triggerid = []

    DAQEvtNumber = []
    ClockCounter = []
    NumTrigLvl2 = []
    StereoEvtNumber = []
    TriggerID = []
    TrigPattern = []
    CalibrationPattern = []
    MacrocellsPattern = []
    TimeDiff = []    
    TrigPatternNames = []
    
    if is_mc == True:
        Energy = [] # TeV
        Core_params = [] # m
        X_max = [] # m
        H_first_int = [] # m
        Shower_angles = []
        
    # valid for runs after 2013-12-13, see MARS MTriggerPatternDecode.cc, ll. 224        
    TrigPatternNames.append('L3 trigger')
    TrigPatternNames.append('Calibration')
    TrigPatternNames.append('LIDAR')
    TrigPatternNames.append('Pedestal')
    TrigPatternNames.append('Pulse injection')
    TrigPatternNames.append('Sumtrigger')
    TrigPatternNames.append('Topo trigger')
    TrigPatternNames.append('L1 trigger')
    
    
    start_time = time.time()

    
    for nevent in range(nevents):

        if nevent >= nevents:
           continue

        sys.stdout.write("\r ... processing event " + str(nevent) + ", " \
                         + "%s >>> Percent Done : %.2f%%" % (datetime.datetime.now(), 100*nevent/float(nevents)))

        Events.GetEvent(nevent)
        
        trigid = evtheader.GetTriggerID()
        
        binarypattern = '\t{0:016b}'.format(trigid&0xff) 
        
        triggpattern = (bool(int(binarypattern[-8])), 
                        bool(int(binarypattern[-1])), 
                        bool(int(binarypattern[-2])), 
                        bool(int(binarypattern[-3])), 
                        bool(int(binarypattern[-4])), 
                        bool(int(binarypattern[-5])), 
                        bool(int(binarypattern[-6])), 
                        bool(int(binarypattern[-7])))
                
        # check mono-character of run:
        #if nevent > 0:
         #   if triggpattern[0] == False:
         #       if StereoEvtNumber[-1] == StereoEvtNumber[-2]:
         #          StereoEvtNumber[-1] = -1
                #else:
                   # print "calib!"
        
        npixels = evtdata_phe.GetNumPixels() + 1
        
        camera.SetCamContent(evtdata_phe)
        camera.SetAllUsed()
        arr_phe_evt = []
        for i_pixel in range(npixels):
            arr_phe_evt.append(camera.GetBinContent(i_pixel + 1))

        camera.SetCamContent(evtdata_arrival)
        camera.SetAllUsed()
        arr_arrival_evt = []
        for i_pixel in range(npixels):
            arr_arrival_evt.append(camera.GetBinContent(i_pixel + 1))
            
        if arr_phe_evt != []:
            arr_phe_2D.append(np.array(arr_phe_evt))
            arr_arrival_2D.append(np.array(arr_arrival_evt))
            
            DAQEvtNumber.append(evtheader.GetDAQEvtNumber())
            ClockCounter.append(evtheader.GetClockCounter())
            NumTrigLvl2.append(evtheader.GetNumTrigLvl2())
            StereoEvtNumber.append(evtheader.GetStereoEvtNumber())
            TriggerID.append(evtheader.GetTriggerID())
            TrigPattern.append(triggpattern)
            CalibrationPattern.append(evtheader.GetCalibrationPattern())
            MacrocellsPattern.append(evtheader.GetMacrocellsPattern())
            TimeDiff.append(evtheader.GetTimeDiff())
            if is_mc == True:
                energy_val = str(mcevtheader.GetEnergyStr())[:-3]
                energy_unit = str(mcevtheader.GetEnergyStr())[-3:]
                if energy_unit == "TeV":
                    Energy.append(float(energy_val))
                elif energy_unit == "GeV":
                    Energy.append(float(energy_val) / 1000.)
                else:
                    raise IOError("Error!")
                Core_params.append((Core_x_vec[nevent] / 100., Core_y_vec[nevent] / 100.)) # meters
                H_first_int.append(h_first_vec[nevent] / 100.) # meters
            
            
    end_time = time.time()
    elapsed = end_time - start_time
    print "\n File read into python arrays >>> Elapsed time: %.1f s" % elapsed
    
    ##########################
    # write data to hdf5 file:
    
    arr_phe_2D = np.array(arr_phe_2D)
    arr_arrival_2D = np.array(arr_arrival_2D)

    
    dsets_events.append(f.create_dataset("dl1/tel"+ str(ntel+1) +"/image", data=arr_phe_2D, compression='gzip'))#, compression_opts=9)
    dsets_events.append(f.create_dataset("dl1/tel"+ str(ntel+1) +"/peakpos", data=arr_arrival_2D, compression='gzip'))#, compression_opts=9)


    # write MC information:
    if is_mc == True:
        dsets_mcheader = []
        
        dt = np.dtype([("Energy", np.double),])
        dsets_mcheader.append(f.create_dataset(os.path.join(dset_name,"McHeader/Energy"), (nevents_check,), dt))
        dsets_mcheader[-1][...] = np.array(Energy, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = "Energy"
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = "TeV"
        
        dt = np.dtype([("Core_x", np.double), ("Core_y", np.double)])
        dsets_mcheader.append(f.create_dataset(os.path.join(dset_name,"McHeader/Core_xy"), (nevents_check,), dt))
        dsets_mcheader[-1][...] = np.array(Core_params, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = "Core_x"
        dsets_mcheader[-1].attrs['FIELD_1_NAME'] = "Core_y"
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = "m"

        dt = np.dtype([("H_first_int", np.double),])
        dsets_mcheader.append(f.create_dataset(os.path.join(dset_name,"McHeader/H_first_int"), (nevents_check,), dt))
        dsets_mcheader[-1][...] = np.array(H_first_int, dtype = dt)
        dsets_mcheader[-1].attrs['FIELD_0_NAME'] = "H_first_int"
        dsets_mcheader[-1].attrs['FIELD_0_UNIT'] = "m"

f.close()

 ... processing event 0, 2018-09-20 17:02:32.683755 >>> Percent Done : 0.00%
 File read into python arrays >>> Elapsed time: 0.0 s


NameError: name 'dset_name' is not defined

In [ ]:
f.close()